# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [64]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape


(27028, 69)


(27028, 69)

Como o número de linhas do sinasc e o número de linhas após ser aplicado o drop_duplicates não foi diferente, podemos afirmar que a base de dados não possuía linhas duplicadas.

In [24]:
# 2) seu código aqui
vetor_missing = sinasc.isnull().sum()


pandas.core.series.Series

o nosso vetor_missing, que é uma série do PANDAS, para cada variável do nosso dataframe, armazena a soma de dados faltantes que se encontram nesta variável, dentro de todo o data frame. Em compensação, como nosso dataframe possui MUITAS variáveis, quando vemos nosso vetor missing por si só talvez não tenhamos noção de quantos dados estão faltantes. Podemos, então, aplicar a função sum no próprio vetor_missing, para fazer esta contagem...

In [26]:
vetor_missing.sum()

121594

Essa quantia, extremamente alta, seria bastante preocupante. No entanto, muitas das nossas variáveis são bastante irrelevantes no que diz respeito à análise da nossa variável de interesse, que é o APGAR-5...

In [108]:
# 3) seu código aqui
# Vamos recriar o nosso dataframe, mas dessa vez considerando apenas as colunas de interesse. Faremos do seguinte modo:
sinasc_reduzido = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_reduzido.shape

(27028, 9)

In [118]:
# 4) seu código aqui
# Como a nossa variável de interesse é o APGAR-5, qualquer linha que tenha o APGAR-5 não preenchido, é completamente irrelevante para nós. Portanto,
# vamos apagar as linhas que possuam o APGAR-5 como missing. Como este novo dataframe também se enquadra como "sinasc_reduzido" e não usaremos o antigo
# sinasc_reduzido para nada, usaremos o inplace = True para excluir completamente os dados droppados.
sinasc_reduzido = sinasc_reduzido.dropna(subset=['APGAR5'])


In [122]:
# 5) seu código aqui
colunas_especificas = ['ESTCIVMAE', 'CONSULTAS']
sinasc_reduzido[colunas_especificas] = sinasc_reduzido[colunas_especificas].replace(9, np.nan)



NESTE TRECHO DO CÓDIGO, fazemos a alteração apenas das colunas específicas por elas mesmas com um replace aplicado, resolvendo o nosso problema, e passando a tratar como NAN o valor 9.

Este trecho de código, está garantindo que os valores que possuem 9 nestas duas variáveis, sejam tratados como NAN, ou seja, MISSING.


In [126]:
# 6) Seu código aqui
# Vamos utilizar o mesmo método que anteriormente... só que agora para substituir os valores faltantes por zero.

sinasc_reduzido['QTDFILVIVO'] = sinasc_reduzido['QTDFILVIVO'].replace(np.nan, 0)
print(sinasc_reduzido['QTDFILVIVO'].isnull().sum())

0


Este código além de trocar os dados faltantes por zero, faz o teste e verifica que, de fato, nenhum valor faltante está presente no nosso dataframe.

In [ ]:
# 7)


Vamos argumentar quanto ao nosso tratamento de dados.
# ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS']

O LOCNASC não é prudente de modo algum, considerar um 'local mais provável', até porque, esta inferência não pode ser feita usando nossos dados. Não existe nenhum motivo sensato para assumirmos que nossos dados não são viciados com relação ao não preenchimento. Na verdade, é bem provável que pessoas que moram em locais específicos muito isolados, não sejam muito mais prováveis de ter este dado faltante do que o 'local mais provável' de nascimento geral dos dados. Portanto, o correto tratamento, suponho, seria o NaN, ou seja, um dado faltante.

IDADAMAE: como, eu suponho, a probabilidade de este dado estar vazio deve ser a mesma idependente da idade da mãe, talvez fosse possível considerar como a média. No entanto, como a idade da mãe possui relação direta com a probabilidade do bebê sobreviver, e consequentemente com o APGAR5, que é a nossa variável de interesse, ao meu ver, o mais sensato seria, sem dúvida, dropar todos os valores faltantes, para garantir que nossa análise não fosse prejudicada, pois certamente a idade média, será uma idade de baixo risco para o bebê, levando-nos a "considerar" que todas as mulheres com idade não documentada estivessem em idade saudável para ter filhos, o que não é nem um pouco sensato. Ou seja, o mais prudente, seria considerar como "dado faltante".

ESTCIVMAE: Presumindo a isonomia da probabilidade de que uma mulher ignore a pergunta sobre seu estado civil (ou seja, independente do estado civil, a chance de ignorar é a mesma, o mais sensato é atribuir ao 9, o valor de um MISSING (eu redigi este texto antes de perceber que já haviamos tratado esta variável, o que, no entanto, aponta que fiz a análise correta...)

ESCMAE: O que posso dizer é que, ao que penso, a probabilidade de a pessoa omitir sua escolaridade deve ser maior, quanto maior for a sua escolaridade. Então, talvez seja sensato, por exemplo, considerar como escolaridade ignorada, a MÉDIA DAS ESCOLARIDADES, DESCONSIDERANDO A CATEGORIA MAIS ALTA... deste modo, consideramos o fato de que é pouquíssimo provável que uma pessoa com mais de 12 anos de escolaridade omitisse sua escolaridade. Como este é um dado categórico, o método mais sensato de calcular esta média é fazendo uma média ponderada da média dos intervalos. Ou seja, pegaremos a quantidade )/de pessoas com 1 a 3 anos de escolaridade, e múltiplicaremos pela média do intervalo (1+3)/2 = 4, depois (4+7)/2, e multiplicar pela quantidade deste intervalo, e assim por diante, e depois dividindo pela quantidade total.

GESTACAO: Supondo a isonomia de probabilidade que a pergunta tenha sido ignorada, o mais adequado seria tomar a mediana dos valores, ou então tomá-lo como missing. Pessoalmente, acho tomar como missing mais apropriado, visto que as probabilidades de mortes são muito diferentes para cada tempo de gestação...

GRAVIDEZ: Aqui, como a probabilidade de a pergunta ser ignorada seja menor no caso de uma gravidez atípica (gêmeos, trigêmeos, ou mais) podemos considerar o ignorado como uma gravidez única...

CONSULTAS: Aqui, eu creio que o mais provavel é que tenha sido ignorado pois a pessoa não sabia ao certo quantas consultas foram realizadas. Ou seja, pouco provável que tenha sido 0. Ou, a pessoa teve vergonha de se julgada por não ter realizado nenhuma consulta, o que também é uma probabilidade... portanto, e também pelo fato de que esta é uma variável muito pertinente para a análise da nossa variável de interesse, o mais sensato é considerar o ignorado como missing.

In [155]:
# Seu código aqui:
sinasc_reduzido['LOCNASC'] = sinasc_reduzido['LOCNASC'].replace(9, np.nan)

print(sinasc_reduzido['ESCMAE'].value_counts())

media_pond_corte = (9*15581+5.5*4727+2*510)/(15581+4727+510)
print(media_pond_corte)
sinasc_reduzido['ESCMAE'] = sinasc_reduzido['ESCMAE'].replace('Nenhuma', '8 a 11 anos')

sinasc_reduzido['GESTACAO'] = sinasc_reduzido['GESTACAO'].replace(9, np.nan)

sinasc_reduzido['GRAVIDEZ'] = sinasc_reduzido['GRAVIDEZ'].replace(9, np.nan)

sinasc_reduzido['CONSULTAS'] = sinasc_reduzido['CONSULTAS'].replace(9, np.nan)



ESCMAE
8 a 11 anos        15621
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Name: count, dtype: int64
8.033792871553464
CONSULTAS
4.0    18587
3.0     5957
2.0     1359
1.0      972
Name: count, dtype: int64


Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.

In [210]:
print(sinasc_reduzido['APGAR5'])

sinasc_reduzido.loc[sinasc_reduzido['APGAR5'] >= 8, 'CAT_APGAR5'] = 'normal'
sinasc_reduzido.loc[(sinasc_reduzido['APGAR5'] <= 7) & (sinasc_reduzido['APGAR5'] >= 6), 'CAT_APGAR5'] = 'asfixia leve'
sinasc_reduzido.loc[(sinasc_reduzido['APGAR5'] >= 4)  & (sinasc_reduzido['APGAR5'] <= 5), 'CAT_APGAR5'] = 'asfixia moderada'
sinasc_reduzido.loc[(sinasc_reduzido['APGAR5'] >= 0) & (sinasc_reduzido['APGAR5'] <= 3), 'CAT_APGAR5'] = 'asfixia severa'
print(sinasc_reduzido['CAT_APGAR5'].value_counts())


0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64
CAT_APGAR5
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [229]:
# 9) seu código aqui
colunas = sinasc_reduzido.columns
print(colunas)
novas_colunas = []
for coluna in colunas:
    novas_colunas.append(coluna.replace(' ', '_').lower())

sinasc_reduzido.columns = novas_colunas
print(sinasc_reduzido)


Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'CAT_APGAR5'],
      dtype='object')
       locnasc  idademae  estcivmae           escmae  qtdfilvivo  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        NaN      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              gestacao gr